In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME ##### ~ Done
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUDv6 import AnimalShelter

###########################
# Data Manipulation / Model
###########################


# Connect to database via CRUD Module ~ Done
db = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned

df = pd.DataFrame.from_records(db.readAll({})) # changed to readAll function to we can pass an empty dictionary

#I'm making a DF for every query type so I can populate the table with only the correct documents
# this allows be to use the dropdown section with update_graphs to update the bar graph
water_df = pd.DataFrame.from_records(db.read({
    "animal_type": "Dog",
    "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
    "sex_upon_outcome": "Intact Female",
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
}))

mountain_df = pd.DataFrame.from_records(db.read({
    "animal_type": "Dog",
    "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
}))
    
disaster_df = pd.DataFrame.from_records(db.read({
    "animal_type": "Dog",
    "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
})) 

allR = [water_df, mountain_df, disaster_df]
allRescue_df = pd.concat(allR)
 
                                 

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

water_df.drop(columns=['_id'],inplace=True)
mountain_df.drop(columns=['_id'],inplace=True)
disaster_df.drop(columns=['_id'],inplace=True)
allRescue_df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__, prevent_initial_callbacks=True)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())




# This function highlights the selected row in the datatable
# Which correlates to the geolocation map.
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_index': row_index },
        'background_color': '#800000'
    } for row_index in selected_rows]



# Display the breeds of animal based on quantity represented in
# the data table
#i added 'all' to the starting var in layout html so graph runs at first without callback needed
@app.callback(
    Output('bar-graph', "figure"),
    [Input('drop-id', "value")],
)
def update_graphs(selected_value):
    ###FIX ME ####
    if selected_value == 'all':
        in_use = df
    elif selected_value == 'combo':
        in_use = allRescue_df
    elif selected_value == 'wr':
        in_use = water_df
    elif selected_value == 'mr':
        in_use = mountain_df
    elif selected_value == 'dr':
        in_use = disaster_df
    else:
        raise preventUpdate
        
    breed_counts = in_use['breed'].value_counts()

    figure = {
        'data': [
            {
                'x': breed_counts.index,
                'y': breed_counts.values,
                'type': 'bar',
                'marker': {
                    'color': breed_counts.values,
                    'colorscale': 'rdgy',
                    'showscale': True,
                }
            }
        ],
        'layout': {
            'title': 'Breed Distribution',
            'xaxis': {'title': 'Breed'},
            'yaxis': {'title': 'Count'}
        }
    }
    return figure


#This function is used to passs the main_layout to app.layout when the reset button is pressed
#Because the main_layout HTML calls the functions 
@app.callback(
    Output('page-content', 'children'),
    [Input('reset_button', 'n_clicks')]
)
def reset_layout(n_clicks):
    if n_clicks is not None and n_clicks > 0:
        # Reset the app layout to the initial state
        return main_layout

    return dash.no_update


#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
    [Input('drop-id', 'value')])
def update_dashboard(selected_value):
## FIX ME Add code to filter interactive data table with MongoDB queries
    if selected_value == 'all':
        return df.to_dict('records')  # Display the original dataframe
    elif selected_value == 'combo':
        return allRescue_df.to_dict('records')  # Display water_df
    elif selected_value == 'wr':
        return water_df.to_dict('records')  # Display water_df
    elif selected_value == 'mr':
        return mountain_df.to_dict('records')  # Display mountain_df
    elif selected_value == 'dr':
        return disaster_df.to_dict('records')  # Display disaster_df
    else:
        raise preventUpdate
    
# This function uses the selected row's data to update the geolocation map
# The marker is kool, red Leaf
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return []  # Return an empty list if there is no data
#code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
 # Because we only allow single row selection, the list can 
 # be converted to a row index here
    if index is None or len(index)==0:
        row = 0
    else: 
        row = index[0]
    #verify name is not null but if it is then print null instead of nothing at all
    animal_name = dff.iloc[row, 9]
    if pd.isnull(animal_name):
        animal_name = "null"
    #I added an icon for the geolocation maker because I wanted it to be red, I tried to just change the color
    # I found that leaflet lets you use their custom leaf icons which have a show so I figured I might aswell 
    icon = {
        "iconUrl": 'https://leafletjs.com/examples/custom-icons/leaf-red.png',
        "shadowUrl": 'https://leafletjs.com/examples/custom-icons/leaf-shadow.png',
        "iconSize": [38, 95],
        "shadowSize": [50, 64],
        "iconAnchor": [22, 94],
        "shadowAnchor": [4, 62],
        "popupAnchor": [-3, -76]
    }
    
# geolocation uses long and lat in row 13 and 14
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
            icon=icon,
                children=[
                    dl.Tooltip(dff.iloc[row,4]),
                    dl.Popup([
                        html.H1("Animal Name:"), 
                        html.H1(animal_name or 'null')    
                    ])
                ]    
            )
        ]
    )
]



#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

main_layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Dog Rescue Candidate Finder', style={'color': 'red'}))),
    html.H3(
        style = {
            'font-size' : '20px',
            'font-family' : 'New Roman',
            'color' : '#808080',
            'text-align' : 'center',
            'text-decoration' : 'overline underline',
            'text-decoration-color' : '#9C1003'
        }, 
        children = 'By Walker Martin'
    ),
    html.Div([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'width': '10%',
                'display': 'inline-block',
                'margin': '0 10px'
            }
        ),
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'width': '15%',
                'display': 'inline-block',
                'margin': '0 10px'
            }
        ),
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'width': '10%',
                'display': 'inline-block',
                'margin': '0 10px'
            }
        )
    ], style={'text-align': 'center'}),
    
    html.Hr(),
    html.Div([
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
#I added a dropbox to change filtering styles per rescue type
    dcc.Dropdown(
        id='drop-id',
        options = [
            {'label' : 'Unfiltered Shelter Data', 'value' : 'all'},
            {'label' : 'All Possible Candidates', 'value' : 'combo'},
            {'label' : 'Mountain/Wilderness Rescue', 'value' : 'mr'},
            {'label' : 'Water Rescue', 'value' : 'wr'},
            {'label' : 'Disaster/Individual Tracking Rescue', 'value' : 'dr'}
        ],
        value = 'all'
    ),
    html.Hr(), 
    html.Button(
        'Reset',
        id='reset_button',
        n_clicks=0,
        style = {'color': 'black',
                 'background-color': 'red',
                 'border': 'none',
                 'padding': '10px',
                 'border-radius': '50%',
                 'cursor': 'pointer',
                 'box-shadow': '0 0 0 2px black, 0 0 0 4px white',
                 'width': '50px',
                 'height': '50px',
                 'display': 'flex',
                 'justify-content': 'center',
                 'align-items': 'center'
            })], className = 'dropdown-container'),
    
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, 
                                   "id": i, 
                                   "deletable": False, 
                                   "selectable": True} for i in df.columns], data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
# I used the code from module 6 here

       #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        row_selectable='single',  # Enable single-row selection
        cell_selectable = True,
        selected_rows=[0],  # Initialize with no selected rows
        page_size=20,  # Set the number of rows displayed
        sort_action='native',  # Enable sorting
        filter_action='native',  # Enable filtering
        style_table={'overflowX': 'scroll'},  # Enable horizontal scrolling
        style_data_conditional=update_styles([0]),
    ), 
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
                style = {'flex': '1'},
                children=[
                    dcc.Graph(
                        id='bar-graph', 
                        figure=update_graphs('all')
                    )
                ]
            ),
            html.Div(
                id = 'map-id',
                className = 'col s12 m6',
                style = {'flex': '1'},
                children = [
                    dcc.Graph(
                        id='map-graph'
                    )
                ]
            )
        ]
    )
])

#dynamic app.layout i order to reset the page
app.layout = html.Div([
    html.Div(id='page-content', children = main_layout)
])

app.run_server(debug=True)

Dash app running on http://127.0.0.1:19184/
